# Decision Tree

## Import Statements

In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, classification_report

## Data Import

In [18]:
df = pd.read_csv('data/preprocessed_data.csv')
df.head()
(df['Label'] == 1).sum() / (df['Label'] == 0).sum()


np.float64(5.987466197364467)

## Extract Features and Labels

In [19]:
X = df.drop(columns=['AdjustedPrice', 'Label']) # Removing price and label features
y = df['AdjustedPrice'] # Predict Price

## Train Val Test Split

In [20]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## Encoding Data

In [21]:
train_features = ['VMakeModel', 'VYear']
eval_features = ['VMakeModel', 'VYear']

# Preprocessing Columns
categorical_cols = ['VMakeModel', 'VMake'] #One Hot Encoding
numerical_cols = ['VYear', 'Distance', 'Months'] # Standardising 

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('scaler', StandardScaler(), numerical_cols)
    ],
    remainder='passthrough'
)

## Create and Training the Model

In [22]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

pipeline.fit(X_train, y_train) # Training pipeline on training data

y_train_pred = pipeline.predict(X_train) # Predicting training data for baseline

## Evaluation

In [ ]:
# Predicting validation data to calculate anomaly threshold
y_val_pred_prices = pipeline.predict(X_val)
residuals = np.abs(y_val - y_val_pred_prices)

# Evaluate on the training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

# Evaluate on the validation set
val_mse = mean_squared_error(y_val, y_val_pred_prices)
val_r2 = r2_score(y_val, y_val_pred_prices)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred_prices)


print("Regression Model Performance")
print(f"Training Mean Squared Error (MSE): {train_mse:.2f}")
print(f"Training R-squared (R²): {train_r2:.2f}")
print(f"Training Mean Absolute Percentage Error (MAPE): {train_mape:.2f}\n")

print(f"Validation Mean Squared Error (MSE): {val_mse:.2f}")
print(f"Validation R-squared (R²): {val_r2:.2f}")
print(f"Validation Mean Absolute Percentage Error (MAPE): {val_mape:.2f}")

# Anomaly Threshold Calculation
anomaly_threshold = np.percentile(residuals, 95) # Threshold determined by 95th percentile
# mean_residual = np.mean(residuals)
# std_residual = np.std(residuals)
# anomaly_threshold = mean_residual + (2 * std_residual)
print(f"Anomaly Threshold: {anomaly_threshold:.2f}")

# Appling anomaly detection on the test data and evaluating
y_test_pred_prices = pipeline.predict(X_test)
y_pred_anomaly_labels = (np.abs(y_test - y_test_pred_prices) > anomaly_threshold).astype(int)

# Using the Labels from the test set for evaluation
y_true_labels = df.loc[y_test.index, 'Label']


--- Regression Model Performance ---
Training Mean Squared Error (MSE): 24748.54
Training R-squared (R²): 0.69
Training Mean Absolute Percentage Error (MAPE): 0.17

Validation Mean Squared Error (MSE): 41351.97
Validation R-squared (R²): 0.50
Validation Mean Absolute Percentage Error (MAPE): 0.20
Anomaly Threshold: 366.82


In [24]:
# Applying the anomaly detection on the test data and evaluating
y_test_pred_prices = pipeline.predict(X_test)
y_pred_anomaly_labels = (np.abs(y_test - y_test_pred_prices) > anomaly_threshold).astype(int)

# Using the Labels from the test set for evaluation
y_true_labels = df.loc[y_test.index, 'Label']

print("Anomaly Detection Evaluation")
print("Confusion Matrix:")
print(confusion_matrix(y_true_labels, y_pred_anomaly_labels))

print("\nClassification Report:")
print(classification_report(y_true_labels, y_pred_anomaly_labels))

Anomaly Detection Evaluation
Confusion Matrix:
[[ 2066   222]
 [13400   591]]

Classification Report:
              precision    recall  f1-score   support

           0       0.13      0.90      0.23      2288
           1       0.73      0.04      0.08     13991

    accuracy                           0.16     16279
   macro avg       0.43      0.47      0.16     16279
weighted avg       0.64      0.16      0.10     16279



Overall model performs poorly as expected, Overfits alot due to not picking up complex details